# Calculates a year in the Jewish calendar

In [24]:
import sys
!{sys.executable} -m pip install zmanim

In [1]:
#defines functions to do math with molads

In [1]:
def mol_remainder_remover(mol):
    while mol[2]>=1080:
        mol[2]=mol[2]-1080
        mol[1]=mol[1]+1
    while mol[1]>=24:
        mol[1]=mol[1]-24
        mol[0]=mol[0]+1
    while mol[0]>7:
        mol[0]=mol[0]-7
    return mol

def mult_mol(mult,mol):
    mol_mult = [mol[0]*mult,mol[1]*mult,mol[2]*mult]
    return mol_remainder_remover(mol_mult)

def add_mol(mol1,mol2):
    mol_summed = [mol1[0]+mol2[0],mol1[1]+mol2[1],mol1[2]+mol2[2]]
    return mol_remainder_remover(mol_summed)

In [2]:
#defines basic data

In [3]:
baharad = [2,5,204]  #ב־ה־ר״ד
monthly_molad = [1,12,793] #א־יב־תשצ״ג
leap_years = [3,6,8,11,14,17,0] #גוחאדז״ט
count_leap = len(leap_years)
machzor_len = 19
count_peshuta = machzor_len - count_leap
months_in_machzor = (len(leap_years)*13)+((19-len(leap_years))*12)
yrp_remain = mult_mol(12,monthly_molad) #ד־ח־תת״ו
yrm_remain = mult_mol(13,monthly_molad) #ה־כא־תקפ״ט
yr_remain = [yrp_remain, yrm_remain]
machzor_remain = add_mol(mult_mol(count_leap,yrm_remain),mult_mol(count_peshuta,yrp_remain)) #ב־יו־תקצ״ה

In [4]:
daysorder = ['א','ב','ג','ד','ה','ו','ז']
daynames = ['Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

In [5]:
daysorder[0]

'א'

In [6]:
months_p1 = ['Tishrei',r'Mar\d{h}eshvan','Kislev','Tevet','Shevat','Adar','Nisan','Iyyar','Sivan','Tammuz','Av','Elul']
months_p2 = ['תשרי','מרחשון','כסלו','טבת','שבט','אדר','ניסן','אייר','סיון','תמוז','אב','אלול']
months_m1 = ['Tishrei',r'Mar\d{h}eshvan','Kislev','Tevet','Shevat','Adar I','Adar II','Nisan','Iyyar','Sivan','Tammuz','Av','Elul']
months_m2 = ['תשרי','מרחשון','כסלו','טבת','שבט','אדר א׳','אדר ב׳','ניסן','אייר','סיון','תמוז','אב','אלול']
months1 = months_p1,months_m1
months2 = months_p2,months_m2
months_hebcal = ['Tishrei', 'Cheshvan', 'Kislev', 'Tevet', 'Shvat', 'Adar1', 'Adar2', 'Nisan', 'Iyyar', 'Sivan', 'Tamuz', 'Av', 'Elul']


In [7]:
def leap_checker(year):
    if year%19 in leap_years:
        return 1
    else:
        return 0

In [8]:
def year_position(yr):
    leap = leap_checker(yr)
    if yr-1 in leap_years:
        motzaei = 1
        erev = 0
    elif yr+1 in leap_years:
        erev = 1
        motzaei = 0
    else:
        erev = 0
        motzaei = 0
    return motzaei, leap, erev

In [9]:
#Calculates info for given year

In [10]:
def tishrei_molad(year):
    machzor=((year-1)//19)+1
    yr_in_machzor = ((year-1)%19)+1
    leap = leap_checker(yr_in_machzor)
    i=1
    q_reg_machzor = 0
    q_leap_machzor = 0
    while i<yr_in_machzor:
        if i in leap_years:
            q_leap_machzor += 1
        else:
            q_reg_machzor += 1
        i+=1
    molsum = [baharad, mult_mol(machzor-1,machzor_remain),mult_mol(q_reg_machzor,yrp_remain),mult_mol(q_leap_machzor,yrm_remain)]
    molad_start = [sum(x) for x in zip(*molsum)]
    molad_start = mol_remainder_remover(molad_start)
    return molad_start,yr_in_machzor

In [11]:
def next_molad(molad,yr):
    leap = leap_checker(yr)
    new_mol = [x + y for x, y in zip(molad, yr_remain[leap])]
    return new_mol

In [12]:
def RH_calc(molad,yr):
    RH1 = molad[0]
    yr_rel = year_position(yr)
    #pushes off for Molad Zaken
    if molad[1]>=18:
        RH1 += 1
    
    #pushes off lo ADU
    adu = [1,4,6]
    if RH1 in adu:
        RH1 += 1
    #prevents a 356-day year by delaying a day
    #applies on moatzaei meuberet when molad is on Tues after 9-204
    #dechiya of ג־ט־ר״ד
    elif yr_rel[1] == 0 and molad[0]==3 and (molad[1]>9 or (molad[1]==9 and molad[2]>=204)):
        RH1 = 5
    #prevents a 383 day year in the previous year by pushing later a day
    #applies in leap year when molad is on Mon after 15-589
    #dechiya of בטותקפ״ד
    elif yr_rel[0] == 1 and molad[0]==2 and (molad[1]>15 or (molad[1]==15 and molad[2]>=589)):
        RH1 = 3
        
    RH1 = ((RH1-1)%7)+1
    return RH1

In [13]:
#display functions
def print_molad(month,molad,leap):
#     print('The Molad of',months1[leap][month],'will be at',molad[0],'days,',molad[1],'hours, and',molad[2],'chalakim')
    print("%",months1[leap][month])
    print(molad[0],"-",molad[1],"-",molad[2])
    
def print_RCh(month,day1,days,leap):
    if days == 2 and day1 == 6:
        print('Rosh Chodesh',months1[leap][month],'will occur on',daynames[day1],'and the following',daynames[(day1+1)%7])
    elif days == 2:
        print('Rosh Chodesh',months1[leap][month],'will occur on',daynames[day1],'and',daynames[day1+1])
    else:
        print('Rosh Chodesh',months1[leap][month],'will occur on',daynames[day1])

In [14]:
def inc_date(date,inc):
    #only works within a month
    date[1] = date[1] + inc
    date[2] = date[2] + inc
    date[2] = (date[2]-1)%7+1
    return date

In [15]:
def new_month(month_len,leap,date,last_RCh,molads):
    RCh_len = month_len[leap][date[0]-1]-28
    RCh_day = inc_date(last_RCh,29)[2]
    date = inc_date(last_RCh,month_len[leap][date[0]])
    
    print_RCh(date[0],RCh_day,RCh_len,leap)
    print_molad(date[0],molads[date[0]],leap)

In [16]:
def daysinyear(year,RH1,RH2):
    if RH1>RH2:
        RH_int = RH2+7-RH1
    else:
        RH_int = RH2-RH1
    #days in year
    leap = leap_checker(year)
    if leap == 0:
        days = 353
    elif leap == 1:
        days = 383
    offset = days%7
    excess = 0
    while offset != RH_int:
        days += 1
        offset = days%7
        excess += 1
    return days,excess

In [17]:
def gregdate(year,month,day):
    months_hebcal = ['Tishrei', 'Cheshvan', 'Kislev', 'Tevet', 'Shvat', 'Adar1', 'Adar2', 'Nisan', 'Iyyar', 'Sivan', 'Tamuz', 'Av', 'Elul']
    leap = leap_checker(year)
    if leap == 0 and month > 6:
        month_ = months_hebcal[month]
    else:
        month_ = months_hebcal[month-1]
    import urllib.request, json, datetime
    with urllib.request.urlopen("https://www.hebcal.com/converter?cfg=json&hy="+str(year)+"&hm="+str(month_)+"&hd="+str(day)+"&h2g=1") as url:
        data = json.loads(url.read().decode())
        #print(data)
    greg_date = datetime.date(data['gy'],data['gm'],data['gd'])
    print(greg_date)

In [18]:
def get_kevia(year):
    year_info = tishrei_molad(year)
    year_molad = year_info[0]
    year_in_machzor = year_info[1]
    RH1 = RH_calc(*year_info)
    next_year_info = tishrei_molad(year+1)
    RH2 = RH_calc(*next_year_info)
    leap = leap_checker(year)
    excess = daysinyear(year,RH1,RH2)[1]
    
    if excess == 0:
        yr_type = '0'
        yr_type2 = 'ח'
    elif excess == 1:
        yr_type = '1'
        yr_type2 = 'כ'
    elif excess == 2:
        yr_type = '2'
        yr_type2 = 'ש'
        
    if excess == 0:
        chesh_len = 29
        kis_len = 29
    elif excess == 1:
        chesh_len = 29
        kis_len = 30
    elif excess == 2:
        chesh_len = 30
        kis_len = 30
    month_lengths_p = [30,chesh_len,kis_len,29,30,29,30,29,30,29,30,29]
    month_lengths_m = [30,chesh_len,kis_len,29,30,30,29,30,29,30,29,30,29]
    month_len = [month_lengths_p,month_lengths_m]
    pesach_day = RH2-2
    if pesach_day < 1:
        pesach_day = pesach_day + 7
    yrcode1 = str(RH1)+yr_type+str(pesach_day)
    yrcode2 = daysorder[RH1-1]+yr_type2+'״'+daysorder[pesach_day-1]
    return [yrcode1,yrcode2]

In [19]:
def run_year(year,mode):
    leap = leap_checker(year)
    year_info = tishrei_molad(year)
    [yrcode1,yrcode2] = get_kevia(year)
    #print basic calendar info--the year and the year-type
#     if leap==0:
#         leap_print = 'Shana Peshuta (non-leap year)'
#     else:
#         leap_print = 'Shana Meubberet (leap year)'
    
    #calculates molads for the year
    molads = [year_info[0]]
    month = 1
    months = 12+leap
    while month<months:
        molads.append(add_mol(molads[month-1],monthly_molad))
        month += 1
    
    #THIS IS FOR KEVIOT TABLE
    if mode == 1:
        yearprint = yrcode1
        if leap == 1:
            yearprint = r"\textit{"+yearprint+"}"
        if year % 7 == 0:
            yearprint = r"\textbf{"+yearprint+"}"
        if year % 10 == 0:
            print(r"\\\hline")
            print(year,"&",yearprint)
        else:
            print("&",yearprint)

    date = [1,1,yrcode1[0]]
    RH_date = date
  
    if mode == 2:
        month_molad = []
        month_molad.append(year_molad)
        print("\multirow{2}{*}{"+str(year)+"}")
        print("&",year_molad[0],"-",year_molad[1],"-",year_molad[2])
        month_counter = 1
        for i in range(1,len(months1[leap])):

            new_molad = add_mol(month_molad[i-1],monthly_molad)
            month_molad.append(new_molad)
            print("&")
            print_molad(i,month_molad[i],leap)
            month_counter += 1
            if leap == 0 and month_counter == 6:
                print(r"&\space\\* \space")
            if leap == 1 and month_counter == 7:
                print(r"\\* \space")
        print(r"&\space\\\hline")

In [20]:
#mode = 1 for keviot
# mode = 2 for molads
year = 5784
run_year(year,1)

& \textit{703}


In [21]:
# year = 5780
# while year < 6000:
#     run_year(year,1)
#     year += 1

In [22]:
#now table with gregorian dates

In [23]:
from zmanim.zmanim_calendar import ZmanimCalendar
from zmanim.astronomical_calendar import AstronomicalCalendar
from zmanim.hebrew_calendar.jewish_calendar import JewishCalendar
from zmanim.util.geo_location import GeoLocation
from zmanim.hebrew_calendar.jewish_date import JewishDate
import urllib.request, json, datetime, time, pytz
import datetime

In [24]:
#https://www.hebcal.com/hebcal?v=1&cfg=json&maj=on&min=off&mod=off&nx=off&yt=H&year=5785&month=x&ss=off&mf=on&c=on&geo=off&M=on&s=off&leyning=off


In [52]:
def year_data(year):
    holidays = ["Rosh Hashana","Yom Kippur","Sukkot I","Chanukah: 2 Candles","Asara B'Tevet","Purim","Pesach I","Shavuot I","Tzom Tammuz","Tish'a B'Av"]
    kevia = get_kevia(year)[0]
    gregyear1 = year-3761
    gregyear2 = year-3760
    gregstring = str(gregyear1)+"-"+str(gregyear2)
    yearinfo = [str(year),gregstring,kevia]
    hebcallink = "https://www.hebcal.com/hebcal?v=1&cfg=json&maj=on&min=off&mod=off&nx=off&yt=H&year="+str(year)+"&month=x&ss=off&mf=on&c=on&geo=off&M=on&s=off&leyning=off"
    with urllib.request.urlopen(hebcallink) as url:
        holiday_info = json.loads(url.read().decode())
    for entry in holiday_info["items"]:
        name = entry["title"]
        if "Erev" not in name and "II" not in name:
            for holiday in holidays:
                if holiday in name:
                    yearinfo.append(datetime.datetime.strptime(entry["date"],"%Y-%m-%d"))
    return yearinfo

In [27]:
def year_range(start_yr,number):
    year_table = []
    for year in range(start_yr,start_yr+number):
        year_table.append(year_data(year))
    return year_table

In [54]:
def make_table(start_yr,number):
    data = year_range(start_yr,number)
    output = []
    output.append(r"\chapter{Selected Holiday's Gregorian Dates}")
    output.append(r"\begin{footnotesize}")
    tabstring = r"\begin{tabular}{c"
    for i in range(1,10):
        tabstring += " | c"
    tabstring += "}"
    output.append(tabstring)
    titles = ["Year:","Gregorian Years","Kevi`a","Rosh Hashana","Yom Kippur","Sukkot",r"\d{H}anukka","10 Tevet","Purim",r"Pesa\d{h}","Shavu`ot","17 Tammuz","9 Av"]
    for i in range(0,len(titles)):
        line = titles[i]
        for j in range(0,number-1):
            item = data[j][i]
            if type(item)==str or type(item) == int:
                line += "&"+str(item)
            else:
                line += "&" + item.strftime("%a %m-%d")
        if i < len(titles):
            line += r"\\\hline"
        else:
            line += r"\\"
        output.append(line)
    output.append(r"\end{tabular}\end{footnotesize}")
    return output

In [56]:
output = make_table(5793,10)
for line in output:
    print(line)

\chapter{Selected Holiday's Gregorian Dates}
\begin{footnotesize}
\begin{tabular}{c | c | c | c | c | c | c | c | c | c}
Year:&5793&5794&5795&5796&5797&5798&5799&5800&5801\\\hline
Gregorian Years&2032-2033&2033-2034&2034-2035&2035-2036&2036-2037&2037-2038&2038-2039&2039-2040&2040-2041\\\hline
Kevi`a&205&723&523&506&113&523&517&225&703\\\hline
Rosh Hashana&Mon 09-06&Sat 09-24&Thu 09-14&Thu 10-04&Mon 09-22&Thu 09-10&Thu 09-30&Mon 09-19&Sat 09-08\\\hline
Yom Kippur&Wed 09-15&Mon 10-03&Sat 09-23&Sat 10-13&Wed 10-01&Sat 09-19&Sat 10-09&Wed 09-28&Mon 09-17\\\hline
Sukkot&Mon 09-20&Sat 10-08&Thu 09-28&Thu 10-18&Mon 10-06&Thu 09-24&Thu 10-14&Mon 10-03&Sat 09-22\\\hline
\d{H}anukka&Thu 09-23&Tue 10-11&Sun 10-01&Sun 10-21&Thu 10-09&Sun 09-27&Sun 10-17&Thu 10-06&Tue 09-25\\\hline
10 Tevet&Sun 11-28&Sat 12-17&Thu 12-07&Wed 12-26&Sun 12-14&Thu 12-03&Wed 12-22&Mon 12-12&Fri 11-30\\\hline
Purim&Sun 12-12&Sun 01-01&Fri 12-22&Thu 01-10&Sun 12-28&Fri 12-18&Thu 01-06&Tue 12-27&Fri 12-14\\\hline
Pesa\d{h}